In [1]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
from dask.distributed import Client

In [3]:
# Path to the dataset
file_path = "sales_data.csv" 

# Load with Pandas using 'Windows-1252' encoding to handle special characters
df_pandas = pd.read_csv(file_path, encoding='Windows-1252')

# Convert to a Dask DataFrame for scalable processing
ddf = dd.from_pandas(df_pandas, npartitions=4)

# Display the first few rows
ddf.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,<NA>,NYC,NY,10022,USA,<NA>,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,<NA>,Reims,<NA>,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,<NA>,Paris,<NA>,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,<NA>,Pasadena,CA,90003,USA,<NA>,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,<NA>,San Francisco,CA,<NA>,USA,<NA>,Brown,Julie,Medium


In [4]:
# Fill missing values in ADDRESSLINE2, STATE, POSTALCODE, and TERRITORY
ddf['ADDRESSLINE2'] = ddf['ADDRESSLINE2'].fillna('Unknown')
ddf['STATE'] = ddf['STATE'].fillna('Unknown')
ddf['POSTALCODE'] = ddf['POSTALCODE'].fillna('Unknown')
ddf['TERRITORY'] = ddf['TERRITORY'].fillna('Unknown')

# Ensure that 'ORDERDATE' is in datetime format
ddf['ORDERDATE'] = dd.to_datetime(ddf['ORDERDATE'], errors='coerce')

# Convert 'SALES' and 'PRICEEACH' to float64 (if not already in correct format)
ddf['SALES'] = ddf['SALES'].astype(float)
ddf['PRICEEACH'] = ddf['PRICEEACH'].astype(float)

# Display cleaned data (showing the first few rows for review)
ddf.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2003-02-24,Shipped,1,2,2003,...,897 Long Airport Avenue,Unknown,NYC,NY,10022,USA,Unknown,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,2003-05-07,Shipped,2,5,2003,...,59 rue de l'Abbaye,Unknown,Reims,Unknown,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,2003-07-01,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,Unknown,Paris,Unknown,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,2003-08-25,Shipped,3,8,2003,...,78934 Hillside Dr.,Unknown,Pasadena,CA,90003,USA,Unknown,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,2003-10-10,Shipped,4,10,2003,...,7734 Strong St.,Unknown,San Francisco,CA,Unknown,USA,Unknown,Brown,Julie,Medium


In [5]:
# Extract the year and month from ORDERDATE
ddf['ORDERYEAR'] = ddf['ORDERDATE'].dt.year
ddf['ORDERMONTH'] = ddf['ORDERDATE'].dt.month

# Calculate total sale (quantity ordered * price each)
ddf['TOTAL_SALE'] = ddf['QUANTITYORDERED'] * ddf['PRICEEACH']

# Display updated dataframe with new features
ddf.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE,ORDERYEAR,ORDERMONTH,TOTAL_SALE
0,10107,30,95.70,2,2871.00,2003-02-24,Shipped,1,2,2003,...,NY,10022,USA,Unknown,Yu,Kwai,Small,2003,2,2871.00
1,10121,34,81.35,5,2765.90,2003-05-07,Shipped,2,5,2003,...,Unknown,51100,France,EMEA,Henriot,Paul,Small,2003,5,2765.90
2,10134,41,94.74,2,3884.34,2003-07-01,Shipped,3,7,2003,...,Unknown,75508,France,EMEA,Da Cunha,Daniel,Medium,2003,7,3884.34
3,10145,45,83.26,6,3746.70,2003-08-25,Shipped,3,8,2003,...,CA,90003,USA,Unknown,Young,Julie,Medium,2003,8,3746.70
4,10159,49,100.00,14,5205.27,2003-10-10,Shipped,4,10,2003,...,CA,Unknown,USA,Unknown,Brown,Julie,Medium,2003,10,4900.00


In [6]:
# Filter data for high-value sales (e.g., SALES > 1000)
ddf_high_sales = ddf[ddf['SALES'] > 1000]

# Show the filtered dataset
ddf_high_sales.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE,ORDERYEAR,ORDERMONTH,TOTAL_SALE
0,10107,30,95.70,2,2871.00,2003-02-24,Shipped,1,2,2003,...,NY,10022,USA,Unknown,Yu,Kwai,Small,2003,2,2871.00
1,10121,34,81.35,5,2765.90,2003-05-07,Shipped,2,5,2003,...,Unknown,51100,France,EMEA,Henriot,Paul,Small,2003,5,2765.90
2,10134,41,94.74,2,3884.34,2003-07-01,Shipped,3,7,2003,...,Unknown,75508,France,EMEA,Da Cunha,Daniel,Medium,2003,7,3884.34
3,10145,45,83.26,6,3746.70,2003-08-25,Shipped,3,8,2003,...,CA,90003,USA,Unknown,Young,Julie,Medium,2003,8,3746.70
4,10159,49,100.00,14,5205.27,2003-10-10,Shipped,4,10,2003,...,CA,Unknown,USA,Unknown,Brown,Julie,Medium,2003,10,4900.00


In [7]:
# Aggregate total sales per customer
total_sales_per_customer = ddf_high_sales.groupby('CUSTOMERNAME')['SALES'].sum().compute()

# Aggregate average sales per order line
average_sales_per_orderline = ddf_high_sales.groupby('ORDERLINENUMBER')['SALES'].mean().compute()

# Count of orders per territory
orders_per_territory = ddf_high_sales.groupby('TERRITORY')['ORDERNUMBER'].count().compute()

# Display aggregated results
print("Total Sales Per Customer:\n", total_sales_per_customer.head())
print("\nAverage Sales Per Order Line:\n", average_sales_per_orderline.head())
print("\nOrders Per Territory:\n", orders_per_territory.head())

Total Sales Per Customer:
 CUSTOMERNAME
AV Stores, Co.             157097.61
Alpha Cognac                70488.44
Amica Models & Co.          93539.66
Anna's Decorations, Ltd    153996.13
Atelier graphique           24179.96
Name: SALES, dtype: float64

Average Sales Per Order Line:
 ORDERLINENUMBER
1    3721.034314
2    3786.589233
3    3834.095811
4    3639.147402
5    3564.428159
Name: SALES, dtype: float64

Orders Per Territory:
 TERRITORY
APAC        217
EMEA       1382
Japan       119
Unknown    1055
Name: ORDERNUMBER, dtype: int64
